In [23]:
FILE = "example3.txt"

In [24]:
from collections import namedtuple
from typing import Literal, get_args, cast
from dataclasses import dataclass, field


Symbol = Literal["|", "-", "L", "J", "7", "F", ".", "S", "X"]

class Point(namedtuple("Point", ["y", "x"])):
    def __add__(self, other):
        return Point(self.y + other.y, self.x + other.x)
    
    def __sub__(self, other):
        return Point(self.y - other.y, self.x - other.x)
        


ADJACENT_COORDINATES = [
    Point(-1,  0),
    Point( 1,  0),
    Point( 0, -1),
    Point( 0,  1)
]

CONNECTS_TO: dict[Symbol, tuple[Point, Point] | tuple] = {
    "|": (Point(-1,  0), Point(1,  0)),
    "-": (Point( 0, -1), Point(0,  1)),
    "L": (Point(-1,  0), Point(0,  1)),
    "J": (Point(-1,  0), Point(0, -1)),
    "7": (Point( 1,  0), Point(0, -1)),
    "F": (Point( 1,  0), Point(0,  1)),
    ".": ()
}

CONNECTED_TO_SYMBOL = {
    tuple(sorted(connected)): symbol
    for symbol, connected in CONNECTS_TO.items()
}

STARTING_CHAR = "S"

@dataclass(eq=True, frozen=True)
class MapTile:
    symbol: Symbol = field(hash=True)
    point: Point = field(hash=True)
    is_loop: bool = field(default=False, hash=False)
    is_added: bool = field(default=False, hash=False)
    is_enclosed: bool = field(default=False, hash=False)

    def set_loop(self, is_loop: bool):
        return MapTile(self.symbol, self.point, is_loop, self.is_added, self.is_enclosed)
    
    def with_point(self, point: Point):
        return MapTile(self.symbol, point, self.is_loop, self.is_added, self.is_enclosed)
    
    def with_enclosed(self, enclosed: bool):
        return MapTile(self.symbol, self.point, self.is_loop, self.is_added, enclosed)
    
    def __eq__(self, __value: object) -> bool:
        if isinstance(__value, MapTile):
            return self.symbol == __value.symbol and self.point == __value.point
        return False
    
    def __str__(self) -> str:
        return str(self.symbol)

    def __repr__(self) -> str:
        return f"MapTile({self.symbol} @ (y={self.point.y}, x={self.point.x}))"


class Map(list[list[MapTile]]):
    def __init__(self, iterable):
        self.min_y = 0
        self.max_y = len(iterable) - 1
        self.min_x = 0
        self.max_x = len(iterable[0]) - 1
        super().__init__(iterable)
    
    def all_tiles(self):
        for row in self:
            for tile in row:
                yield tile

    def __getitem__(self, index):
        if isinstance(index, Point):
            y, x = index
            return self[y][x]
        return super().__getitem__(index)
    
    def __setitem__(self, index, value):
        print(index, value)
        if isinstance(index, Point):
            y, x = index
            row = self[y]
            row[x] = value
            return
        super().__setitem__(index, value)
    
    def point_valid(self, point: Point) -> bool:
        return (self.min_y <= point.y <= self.max_y) and (self.min_x <= point.x <= self.max_x)
    
    def get_adjacent(self, tile: MapTile) -> list[MapTile]:
        point = tile.point
        adjacent = [point + adj for adj in ADJACENT_COORDINATES]
        adjacent = [point for point in adjacent if self.point_valid(point)]
        adjacent_tiles = [self[point] for point in adjacent]
        return adjacent_tiles
    
    def possibly_connected(self, tile: MapTile) -> set[MapTile]:
        point = tile.point
        connected_coords = [point + coords for coords in CONNECTS_TO[tile.symbol]] # type: ignore
        connected_coords = [point for point in connected_coords if self.point_valid(point)]
        return set(self[point] for point in connected_coords)
        
    def get_connected(self, tile: MapTile) -> list[MapTile]:
        connected = []
        tile_connects_to = self.possibly_connected(tile)
        for other_tile in tile_connects_to:
            other_connected_to = self.possibly_connected(other_tile)
            if tile in other_connected_to:
                connected.append(other_tile)
        return connected
    
    def connected(self, tile1: MapTile, tile2: MapTile) -> bool:
        tile1_connects_to = self.possibly_connected(tile1)
        if tile2 in tile1_connects_to:
            tile2_connects_to = self.possibly_connected(tile2)
            return tile1 in tile2_connects_to
        return False

    def calculate_enclosed(self, tile: MapTile) -> MapTile:
        if tile.is_loop or tile.is_added:
            return tile.with_enclosed(False)
            
        def is_exit(tile: MapTile) -> bool:
            y, x = tile.point
            return not tile.is_loop and x == self.min_x or x == self.max_x or y == self.min_y or y == self.max_y

        def can_enter(tile: MapTile) -> bool:
            return not tile.is_loop

        adjacent = self.get_adjacent(tile)
        adjacent_can_enter = [tile for tile in adjacent if can_enter(tile)]
        visited = set()
        to_visit = adjacent_can_enter

        while len(to_visit) != 0:
            # print([x.point for x in to_visit])
            tile_to_visit, *to_visit = to_visit
            if tile_to_visit not in visited:
                if is_exit(tile_to_visit):
                    return tile.with_enclosed(False)

                adjacent = self.get_adjacent(tile_to_visit)
                adjacent_can_enter = [tile for tile in adjacent if can_enter(tile)]

                for adjactent_tile in adjacent_can_enter:
                    if adjactent_tile not in visited:
                        to_visit.append(adjactent_tile)

                visited.add(tile_to_visit)

        return tile.with_enclosed(True)
        

In [25]:
def ensure_is_symbol(char: str) -> Symbol:
    possible_values = get_args(Symbol)
    if char in possible_values:
        return cast(Symbol, char)
    raise ValueError(f"Wrong character: {char}. Should be one of {possible_values}")

def parse_map(data: str) -> Map:
    return Map(
        [
            [
                MapTile(ensure_is_symbol(char), Point(y, x)) for x, char in enumerate(list(row))
            ]
            for y, row in enumerate(data.splitlines())
        ]
    )

def print_map(map: Map):
    for row in map:
        print("[", *row, "]")

In [26]:
with open(f"10/{FILE}") as f:
    data = f.read()
map = parse_map(data)

In [27]:
print_map(map)

[ 7 - F 7 - ]
[ . F J | 7 ]
[ S J L L 7 ]
[ | F - - J ]
[ L J . L J ]


## Coordinates
Coordinates are assumed to be (y, x) with (0, 0) in the bottom left, y increasing down, and x increasing right.

In [28]:
def find_starting_point(map: Map) -> MapTile:
    for tile in map.all_tiles():
        if tile.symbol == STARTING_CHAR:
            return tile
    raise ValueError(f"Incorrect map, no starting char: {STARTING_CHAR}")

In [29]:
def starting_tile_type(map: Map):
    starting_tile = find_starting_point(map)
    adjacent_to_start = map.get_adjacent(starting_tile)
    connected_to_start: list[MapTile] = []
    for tile in adjacent_to_start:
        possibly_connected = map.possibly_connected(tile)
        if starting_tile in possibly_connected:
            connected_to_start.append(tile)
    coord_diffs = [tile.point - starting_tile.point for tile in connected_to_start]
    start_type = CONNECTED_TO_SYMBOL[tuple(sorted(coord_diffs))]
    return MapTile(start_type, starting_tile.point, True, starting_tile.is_added)

starting_tile_replacement = starting_tile_type(map)
map[starting_tile_replacement.point] = starting_tile_replacement

Point(y=2, x=0) F


In [30]:
starting_tile = starting_tile_replacement
connected_to_start = map.get_connected(starting_tile)

In [31]:
visited = {starting_tile: 0}
to_visit = [(1, connected) for connected in connected_to_start]

while len(to_visit) != 0:
    (distance, tile), *to_visit = to_visit
    if tile not in visited:
        visited[tile] = distance
        map[tile.point] = tile.set_loop(True)
        connected = map.get_connected(tile)
        for connected_tile in connected:
            if connected_tile not in visited:
                to_visit.append((distance+1, connected_tile))

Point(y=2, x=1) J
Point(y=3, x=0) |
Point(y=1, x=1) F
Point(y=4, x=0) L
Point(y=1, x=2) J
Point(y=4, x=1) J
Point(y=0, x=2) F
Point(y=3, x=1) F
Point(y=0, x=3) 7
Point(y=3, x=2) -
Point(y=1, x=3) |
Point(y=3, x=3) -
Point(y=2, x=3) L
Point(y=3, x=4) J
Point(y=2, x=4) 7


In [32]:
def print_visited(map: Map, visited: dict[MapTile, int]):
    for row in map:
        vis = []
        for tile in row:
            if tile in visited:
                vis.append(visited[tile])
            else:
                vis.append(".")
        print("[", *vis, "]")

In [33]:
visited

{MapTile(F @ (y=2, x=0)): 0,
 MapTile(J @ (y=2, x=1)): 1,
 MapTile(| @ (y=3, x=0)): 1,
 MapTile(F @ (y=1, x=1)): 2,
 MapTile(L @ (y=4, x=0)): 2,
 MapTile(J @ (y=1, x=2)): 3,
 MapTile(J @ (y=4, x=1)): 3,
 MapTile(F @ (y=0, x=2)): 4,
 MapTile(F @ (y=3, x=1)): 4,
 MapTile(7 @ (y=0, x=3)): 5,
 MapTile(- @ (y=3, x=2)): 5,
 MapTile(| @ (y=1, x=3)): 6,
 MapTile(- @ (y=3, x=3)): 6,
 MapTile(L @ (y=2, x=3)): 7,
 MapTile(J @ (y=3, x=4)): 7,
 MapTile(7 @ (y=2, x=4)): 8}

In [34]:
print_visited(map, visited)

[ . . 4 5 . ]
[ . 2 3 6 . ]
[ 0 1 . 7 8 ]
[ 1 4 5 6 7 ]
[ 2 3 . . . ]


In [35]:
max(visited.values())

8

In [36]:
adjacent = map.get_adjacent(starting_tile)
adjacent

[MapTile(. @ (y=1, x=0)), MapTile(| @ (y=3, x=0)), MapTile(J @ (y=2, x=1))]

In [37]:
map.connected(map[2][1], map[3][1])

False

In [38]:
print_map(map)

[ 7 - F 7 - ]
[ . F J | 7 ]
[ F J L L 7 ]
[ | F - - J ]
[ L J . L J ]


In [39]:
from itertools import pairwise

def expand_horizontally(map: Map):
    new_rows = []
    for row in map:
        new_row = []
        for tile1, tile2 in pairwise(row):
            if tile1.is_loop and tile2.is_loop and map.connected(tile1, tile2):
                new_point_is_loop = True
                new_point_symbol = "-"
            else:
                new_point_is_loop = False
                new_point_symbol = "X"
            
            new_x = len(new_row)
            new_first_point = tile1.with_point(Point(tile1.point.y, new_x))
            new_middle_point = MapTile(new_point_symbol, Point(tile1.point.y, new_x + 1), is_loop=new_point_is_loop, is_added=True)
            
            new_row.extend([new_first_point, new_middle_point])
        last_tile = row[-1]
        new_last_tile = last_tile.with_point(Point(last_tile.point.y, len(new_row)))
        new_row.append(new_last_tile)
        new_rows.append(new_row)
    return Map(new_rows)

def expand_vertically(map: Map):
    new_rows = []
    for row1, row2 in pairwise(map):
        new_middle_row = []
        new_y = len(new_rows) + 1
        for new_x, (tile1, tile2) in enumerate(zip(row1, row2)):
            if tile1.is_loop and tile2.is_loop and map.connected(tile1, tile2):
                new_point_is_loop = True
                new_point_symbol = "|"
            else:
                new_point_is_loop = False
                new_point_symbol = "X"
            
            new_tile = MapTile(new_point_symbol, Point(new_y, new_x), is_loop=new_point_is_loop, is_added=True)
            new_middle_row.append(new_tile)
        
        first_row_y = len(new_rows)
        new_first_row = [tile.with_point(Point(first_row_y, tile.point.x)) for tile in row1]
        new_rows.extend([new_first_row, new_middle_row])

    last_row = map[-1]
    new_last_y = len(new_rows)
    new_last_row = [tile.with_point(Point(new_last_y, tile.point.x)) for tile in last_row]
    new_rows.append(new_last_row)

    return Map(new_rows)


In [40]:
new_map = expand_horizontally(map)
new_map = expand_vertically(new_map)
print_map(new_map)

[ 7 X - X F - 7 X - ]
[ X X X X | X | X X ]
[ . X F - J X | X 7 ]
[ X X | X X X | X X ]
[ F - J X L X L - 7 ]
[ | X X X X X X X | ]
[ | X F - - - - - J ]
[ | X | X X X X X X ]
[ L - J X . X L X J ]


In [41]:
def print_enclosed(map: Map):
    for row in map:
        vis = []
        for tile in row:
            if tile.is_enclosed:
                vis.append("I")
            else:
                vis.append(tile.symbol)
        print("[", *vis, "]")

In [42]:
from tqdm import tqdm
all_tiles = len(new_map) * len(new_map[0])
num_rows = len(new_map)

new_rows = []
for row in tqdm(new_map):
    new_row = [new_map.calculate_enclosed(tile) for tile in row]
    new_rows.append(new_row)

enclosed_map = Map(new_rows)

100%|██████████| 9/9 [00:00<00:00, 30840.47it/s]


In [43]:
sum(1 if tile.is_enclosed else 0 for tile in enclosed_map.all_tiles())

1